In [1]:
import Task_two as two
import bn256
import numpy as np
import random
from PIL import Image


In [2]:
### вектор, який будується з фото використовується в якості регістрації
### вектор для аутентефікації будується рандомно

In [3]:
def reg_vec_image(fotoname):
    img = Image.open(fotoname)
    gray = img.convert('L')
    bw = gray.point(lambda x: x)
    data = np.array(bw.getdata())
    data = np.where(data >= 128, 1, -1)
    return data


In [4]:
v1 = reg_vec_image("eye_for_4 task.jpg")

In [5]:
import functools

def benchmark(f):
    
    import time
    @functools.wraps(f)             
    def _benchmark(*args, **kw):
        t =time.perf_counter()
        rez = f(*args, **kw)
        t = time.perf_counter() - t        
        print('{0} time elapsed {1:.8f} s.'.format(f.__name__, t))
        return rez
    return _benchmark

In [6]:
len_v1_or_v2 = len(v1)

In [7]:
v2 = np.random.choice([1, -1], len_v1_or_v2)


In [8]:
v1, v2

(array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1, -1,
        -1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1,  1, -1,
        -1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,
         1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
        -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,
         1, 

In [9]:
hm = two.hamming_distance(v1, v2)

In [10]:
G1 = bn256.curve_G
G2 = bn256.twist_G

In [11]:
v1points = [G2.scalar_mul(bn256.order + i) for i in v1]
v2points  = [G1.scalar_mul(bn256.order + i) for i in v2]

In [12]:
def building_keys():
    s = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2)]
    t = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2)]
    u = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2 + 2)]
    v = [random.randrange(2, bn256.order) for _ in range(len_v1_or_v2 + 2)]
    h1 = random.randrange (2, bn256.order)
    h2 = random.randrange (2, bn256.order)
    H1 = G1.scalar_mul(h1)
    H2 = G2.scalar_mul(h2)
    gen1_h = [G1.scalar_mul(int(s[i])).add(H1.scalar_mul(int(t[i]))) for i in range(len(s))]
    gen2_h = [G2.scalar_mul(int(u[i])).add(H2.scalar_mul(int(v[i]))) for i in range(len(u))]
    return [[G2, H2, s, t, gen2_h], [G1, H1, u, v, gen1_h]]
    

In [13]:
msk = building_keys()

In [14]:
msk

[[((...))]],
 [(1, ...)]]]

In [15]:
@benchmark    
def IPE_Registration(msk, v1points):
    
    r0 = random.randrange(2, bn256.order)
    
    reg_template = []
    reg_template.append(msk[0].scalar_mul(r0))
    reg_template.append(msk[1].scalar_mul(r0))
    for i in range(len_v1_or_v2 + 2):
        reg_template.append(msk[4][i].scalar_mul(r0))
         
    for i in range(len_v1_or_v2):
        reg_template[2] = reg_template[2].add(v1points[i].scalar_mul(msk[2][i]).scalar_mul(bn256.order - 1))
        reg_template[3] = reg_template[3].add(v1points[i].scalar_mul(msk[3][i]).scalar_mul(bn256.order - 1))
        reg_template[i+4] = reg_template[i+4].add(v1points[i])
    
    
    return reg_template

    

In [16]:
result_reg = IPE_Registration(msk[0], v1points)

IPE_Registration time elapsed 229.70966745 s.


In [17]:
result_reg

[((...))]

In [18]:
@benchmark    
def IPE_Authentication(msk, v2points ):
    r0 = random.randrange(2, bn256.order)
#     print(r0)
    auth_template = []
    for i in range(2):
        auth_template.append(msk[0].scalar_mul(0))
    auth_template.append(msk[0].scalar_mul(r0))
    auth_template.append(msk[1].scalar_mul(r0))
    for i in range(len_v1_or_v2):
        auth_template.append(msk[4][i].scalar_mul(r0).add(v2points[i]))
    for i in range(len_v1_or_v2 + 2):
        auth_template[0] = auth_template[0].add((auth_template[i+2].scalar_mul(msk[2][i])).scalar_mul(bn256.order - 1))
        auth_template[1] = auth_template[1].add(auth_template[i+2].scalar_mul(msk[3][i]).scalar_mul(bn256.order - 1))
    return auth_template

In [19]:
result_aut = IPE_Authentication(msk[1], v2points )

IPE_Authentication time elapsed 56.94226281 s.


In [20]:
result_aut

[(...)]

In [21]:
@benchmark    
def building_op_ate_e():
    e = bn256.optimal_ate(result_reg[0], result_aut[0])
    for i in range(1, len_v1_or_v2 + 4):
        e = e.mul(bn256.optimal_ate(result_reg[i], result_aut[i]))
    return e
    

In [22]:
e = building_op_ate_e()

building_op_ate_e time elapsed 132.81512835 s.


In [23]:
e

(((28083006659551315010722791427300506528190550245536748096374848831470352420371,54519568047787097028618328902862058975241659538161673197212552024448354503900),(3090797703326428129124098882749467603940302050230518106308856253238816406135,7028071121980934559053072312634647634511668938743222872786543236791703790574),(58006133078076235628009504729964524754330830123045645332365500049953299454642,22161168349666707928400271642447957247099145954040991623157789434796514754830)),((16214297498799860640662073518475228880897726244541316948796013867971793583470,59545848676959111701722486239756499226779978971740182369770161538253380568086),(22614083019736058540112374772474972150174944275577033028451483053454878148820,578869513352277384492135292376512937912486475988217450427191663315014761565),(50461140834256330771275628655200305629588019819388410447116601469822170353965,11979663406512329320155889263495037235813250904588438763858008245628566118962)))

In [24]:
@benchmark    
def building_logtable():
    spar_one = bn256.optimal_ate(G2, G1)
    log_table = [spar_one.exp(i) for i in range(len_v1_or_v2, -1, -2)]
    spar_two = bn256.optimal_ate(G2, G1.scalar_mul(bn256.order - 1))
    log_table = log_table + [spar_two.exp(i) for i in range(2, len_v1_or_v2 + 1, 2)]
    return log_table

In [25]:
log_table = building_logtable()

building_logtable time elapsed 4.26450860 s.


In [26]:
log_table

[(((...))),
 (((0,0),(0,0),(0,0)),((0,0),(0,0),(0,1))),
 (((...)))]

In [27]:
index_e_log = building_logtable().index(e)

building_logtable time elapsed 4.25389207 s.


In [28]:
log_table.index(e)

218

In [29]:
hm == index_e_log

True